In [1]:
import numpy as np
import pandas as pd
import yfinance as yf

## Download Data

In [2]:
import os

cwd = os.getcwd()
# Check if price data have been saved
price_data = os.path.join(cwd, "price_data")
if not os.path.exists(price_data):
    # Getting the Nasdaq 100 ticker symbols from Wikipedia
    nas_df = pd.read_html("https://en.wikipedia.org/wiki/Nasdaq-100")[4]
    tickers = nas_df.Ticker.to_list()
    # Download the Nasdaq 100 Adjusted Close price data with yfinance
    df = yf.download(tickers, start="2010-01-01")
    # Create price_data directory
    os.makedirs(price_data)
    # Save a copy of the dataframe to price_data directory
    df.to_csv(os.path.join(price_data, "Nasdaq-100.csv"))
else:
    # Load the Nasdaq-100.csv
    df = pd.read_csv(os.path.join(price_data, "Nasdaq-100.csv"), 
                     header=[0,1], index_col=0)
    df.index = pd.to_datetime(df.index)

In [3]:
df = df.swaplevel(axis=1)

## Percent Change Calculation

In [5]:
def ATR(data, p=14):
    hl = data['High'] - data['Low']
    hc = (data['High'] - data['Close'].shift(1)).abs()
    lc = (data['Low'] - data['Close'].shift(1)).abs()
    tr = pd.concat([hl, hc, lc], axis=1).max(1)
    return tr.rolling(p).mean()

In [230]:
data = df['AMD'].dropna()

In [231]:
adj_close = data['Adj Close']
buy_and_hold = adj_close.iloc[-1] / adj_close.iloc[0] - 1
max_profit = adj_close.max() / adj_close.iloc[0] - 1

print(f'Return while buy and hold: {buy_and_hold:.2f} times')
print(f'Return while sold at max high: {max_profit:.2f} times')

Return while buy and hold: 6.12 times
Return while sold at max high: 15.69 times


In [232]:
data['atr'] = ATR(data)
sma = data['Close'].rolling(200).mean()
mask = data['Close'] < sma
data['can_buy'] = mask.shift(1)

In [233]:
data = data.dropna()

In [239]:
holding = False
buy_at = 0
gain = []
tp = 0
sl = 0

for i in range(len(data.iloc[-500:])):
    da = data.iloc[i]
    if not holding and da['can_buy']:
        buy_at = da['Close']
        tp = buy_at + (da['atr'] * 4)
        sl = buy_at - (da['atr'] * 1)
        holding = True
    else:
        sell_at = 0
        if da['Open'] < sl:
            sell_at = da['Open']
        elif da['High'] > tp:
            sell_at = tp
        elif da['Low'] < sl:
            sell_at = sl
        
        if sell_at != 0:
            gain.append(sell_at / buy_at)
            holding = False
            continue

ret = np.array(gain)
if len(ret) > 0:
    print(f"return: {ret.prod() - 1:.2f} x")
    print(f"num trades: {len(ret)}")
    print(f"win rate: {(ret > 1).mean()*100:.2f}%")
    print(f"trade rate: {len(ret)/len(data)*100:.2f}%")
else:
    print("No trades")

return: 955.40 x
num trades: 65
win rate: 78.46%
trade rate: 1.99%
